In [3]:
import ExonExonData as eed
import pathlib
cwd = pathlib.Path.cwd()
import pandas
import numpy
import pickle
from Heatmaps import heat_embedding
from Heatmaps import heatmapv2 as heatmap
from Heatmaps import _undigitize_seq as undigit


In [ ]:
data_file = cwd.parent / "Data_Files" / "Primates" / "Genetics" / "Homo_sapiens" / "Cleaned_Data" / "Data_Set_2_frame.pkl"
transform = numpy.log2

In [ ]:
_, exon, intron, _, _, exon_max, exon_min, intron_max, intron_min = heat_embedding(data_file,
                                                                                    # n = n,
                                                                                    # k_m = k, k_p = k,
                                                                                    just_import = True,
                                                                                    log_transform = transform)  # Just import is turned on because I have one dictionary with all the data. I'll use that one set for all the images from now on. Just makes things a little faster and easier.
